In [1]:
# Imports
import os
os.chdir("/projectnb/labci/Lucia/edu-bayesian-MCMC/")

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pymc as pm
from pymc import PolyaGamma as PG
import statsmodels.api as sm
from statsmodels.graphics import tsaplots

from src.samplers.MetropolisGibbs import gibbs
from src.samplers.PolyaGamma import polyagamma

In [3]:
# Simulate data
I = 100   # exam items
P = 1000  # pupils

init_a = torch.zeros(I)  
init_b = torch.zeros(I) 
init_t = torch.zeros(P) 
init_W = torch.zeros(P, I)


true_a = torch.empty(size=(I,))      # items' discriminatory power
true_b = torch.empty(size=(I,))      # items' difficulty
true_theta = torch.empty(size=(P,))  # students' skills
true_W = torch.empty(size=(P,I))     # latent variable (for PG case)
Y = torch.empty(size=(P, I))         # simulated data

# populate a, b:
for i in range(I):
    U = torch.distributions.Uniform(torch.tensor([0.0]), torch.tensor([1.0]))
    true_a[i] = U.sample()
    true_b[i] = -1.0*U.sample()
    
# populate theta:
for p in range(P):
    U = torch.distributions.Uniform(torch.tensor([0.0]), torch.tensor([1.0]))
    true_theta[p] = U.sample()

# populate W:
scales = torch.mul(torch.t(true_a[:, None]), true_theta[:, None]) + true_b
true_W = torch.from_numpy(pm.draw(PG.dist(h=1, z=scales)))

# generate exam data:
for i in range(I):
    for p in range(P):
        prob = torch.exp(true_a[i]*true_theta[p] + true_b[i]) / (1 + torch.exp(true_a[i]*true_theta[p] + true_b[i]))
        B = torch.distributions.Bernoulli(prob)
        Y[p][i] = B.sample()

#Y = Y.to(device)

In [6]:
# Run samplers
N = 100
sigmasq_a, sigmasq_b, sigmasq_t = torch.tensor([1.0]), torch.tensor([1.0]), torch.tensor([1.0])  # priors

samples_mwg = gibbs(init_a, init_b, init_t, Y, sigmasq_a, sigmasq_b, sigmasq_t, niter=N, adapt=False)
print("")
samples_ada_mwg = gibbs(init_a, init_b, init_t, Y, sigmasq_a, sigmasq_b, sigmasq_t, niter=N, adapt=True)
print("")
samples_pg = polyagamma(init_a, init_b, init_t, init_W, Y, sigmasq_a, sigmasq_b, sigmasq_t, niter=N)

Starting Gibbs sampler... 
--------------------------------------------



  0%|          | 0/99 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument std in method wrapper_CUDA_Tensor_Tensor_normal)

### Plots

In [ ]:
fig1, axs = plt.subplots(nrows=3, ncols=3, figsize=(9,9), sharex=True)
fig1.suptitle("Traceplots for MCMC samples of a, b, theta under 3 sampling schemes")

t = range(N)
data_to_plot = [samples_mwg[0][:,0], samples_ada_mwg[0][:,0], samples_pg[0][:,0],
                samples_mwg[1][:,0], samples_ada_mwg[1][:,0], samples_pg[1][:,0],
                samples_mwg[2][:,0], samples_ada_mwg[2][:,0], samples_pg[2][:,0]]

true_values = [true_a[0], true_a[0], true_a[0],
               true_b[0], true_b[0], true_b[0],
               true_theta[0], true_theta[0], true_theta[0]]

cols = ["Metropolis within Gibbs", "Adaptive MwG", "Gibbs (Polya-Gamma)"]
rows = ["a[0]", "b[0]", "theta[0]"]

for i, ax in enumerate(fig1.axes):
    ax.plot(t, data_to_plot[i])
    ax.axhline(true_values[i], color="red")

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size="small")

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90)

fig1.supxlabel('iterations')
fig1.tight_layout()
plt.show()

In [ ]:
fig2, axs = plt.subplots(nrows=3, ncols=3, figsize=(9,9))
fig2.suptitle("Histograms of MCMC samples of a, b, theta under 3 sampling schemes")

cols = ["Metropolis within Gibbs", "Adaptive MwG", "Gibbs (Polya-Gamma)"]
rows = ["a[0]", "b[0]", "theta[0]"]

for i, ax in enumerate(fig2.axes):
    ax.hist(data_to_plot[i], bins=100)
    ax.axvline(true_values[i], color="red")

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size="small")

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90)

fig2.tight_layout()
plt.show()

In [ ]:
fig3, axs = plt.subplots(nrows=3, ncols=3, figsize=(9,9), sharey=True)
fig3.suptitle("ACF for MCMC samples of a, b, theta under 3 sampling schemes")

cols = ["Metropolis within Gibbs", "Adaptive MwG", "Gibbs (Polya-Gamma)"]
rows = ["a[0]", "b[0]", "theta[0]"]

for i, ax in enumerate(fig3.axes):
    tsaplots.plot_acf(data_to_plot[i], lags=50, ax=ax, title="")

for ax, col in zip(axs[0], cols):
    ax.set_title(col, size="small")

for ax, row in zip(axs[:,0], rows):
    ax.set_ylabel(row, rotation=90)

fig3.supxlabel('lag')
fig3.tight_layout()
plt.show()

### Accuracy

In [ ]:
def mse(chain, true_val):
    MSE = nn.MSELoss()
    E = torch.empty(size=(len(chain),))

    for s in range(len(chain)):
        E[s] = MSE(chain[s], true_val)

    return E

In [ ]:
fig4, axs = plt.subplots(nrows=3, ncols=1, figsize=(9,9), sharex=True)
fig4.suptitle("MSE of MCMC samples of a, b, theta under 3 sampling schemes over time")

t = range(N)
errors_to_plot = [[mse(samples_mwg[0], true_a), mse(samples_ada_mwg[0], true_a), mse(samples_pg[0], true_a)],
                  [mse(samples_mwg[1], true_b), mse(samples_ada_mwg[1], true_b), mse(samples_pg[1], true_b)],
                  [mse(samples_mwg[2], true_theta), mse(samples_ada_mwg[2], true_theta), mse(samples_pg[2], true_theta)]]

colors = ["red", "blue", "green"]
labels = ["Metropolis within Gibbs", "Adaptive MwG", "Gibbs (Polya-Gamma)"]
rows = ["a", "b", "theta"]

for i, ax in enumerate(fig4.axes):
    lines = data_to_plot[i]
    
    for j in range(len(lines)):
        ax.plot(t, lines[j], color=colors[j], label=labels[j])

for ax, row in zip(axs, rows):
    ax.set_ylabel(row, rotation=90)

axs[0].legend(loc="best")
fig4.supxlabel('iterations')
fig4.tight_layout()
plt.show()

In [ ]:
for i in range(len(errors_to_plot)):
    for j in range(len(errors_to_plot[i])):
        mean = torch.mean(errors_to_plot[i][j])
        print(f"Overall MSE for variable {rows[i]} under {labels[j]} scheme: {mean:.3f}")

### Asymptotic Variance